# Web content text tokenizer 

Splits titles from web articles into verbs, adjectives, adverbs, superlatives and named entities. Outputs the results to Excel.

In [1]:
import pandas as pd
import numpy as np

# to ignore the warning
import warnings
warnings.filterwarnings('ignore')

import re

In [1]:
!pip install spacy==2

  Using cached spacy-2.0.0.tar.gz (13.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached murmurhash-0.28.0.tar.gz (23 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached cymem-1.31.2.tar.gz (33 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached preshed-1.0.1.tar.gz (112 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached thinc-6.10.3.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached plac-0.9.6-py2.py3-none-any.whl (20 kB)
  Using cached pathlib-1.0.1-py3-none-any.whl (14 kB)
  Using cached dill-0.2.9-py3-none-any.whl
  Using cached regex-2017.04.05.tar.gz (601 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (set

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [536 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\spacy
  copying spacy\about.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\compat.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\glossary.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\language.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\lemmatizer.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\scorer.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\util.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\_ml.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\__init__.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\__main__.py -> build\lib.win-amd64-3.8\spacy
  creating build\lib.win-amd64-3.8\spacy\cli
  copying spacy\cli\convert.py -> build\lib.win-amd64-3.8\spacy\cli
  

  copying spacy\tests\tokenizer\test_naughty_strings.py -> build\lib.win-amd64-3.8\spacy\tests\tokenizer
  copying spacy\tests\tokenizer\test_tokenizer.py -> build\lib.win-amd64-3.8\spacy\tests\tokenizer
  copying spacy\tests\tokenizer\test_urls.py -> build\lib.win-amd64-3.8\spacy\tests\tokenizer
  copying spacy\tests\tokenizer\test_whitespace.py -> build\lib.win-amd64-3.8\spacy\tests\tokenizer
  copying spacy\tests\tokenizer\__init__.py -> build\lib.win-amd64-3.8\spacy\tests\tokenizer
  creating build\lib.win-amd64-3.8\spacy\tests\vectors
  copying spacy\tests\vectors\test_similarity.py -> build\lib.win-amd64-3.8\spacy\tests\vectors
  copying spacy\tests\vectors\test_vectors.py -> build\lib.win-amd64-3.8\spacy\tests\vectors
  copying spacy\tests\vectors\__init__.py -> build\lib.win-amd64-3.8\spacy\tests\vectors
  creating build\lib.win-amd64-3.8\spacy\tests\vocab
  copying spacy\tests\vocab\test_add_vectors.py -> build\lib.win-amd64-3.8\spacy\tests\vocab
  copying spacy\tests\vocab\te

In [2]:
input_filename  = 'data/source/find-keywords.xlsx'
output_filename = 'data/find-keywords-nouns.xlsx'

Load the articles and display a sample of the data

In [3]:
df = pd.read_excel(input_filename)
print('Article count: ', len(df))
# For testing
# df = df.head(10)
df.head(10)

Article count:  593


,Url,Title,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Abstract
0,NaN,ParaDocs provides our clients with the highest...,NaN,NaN,NaN,NaN,NaN
1,NaN,"AGNITY Global Inc., (AGNITY) is a global provi...",NaN,NaN,NaN,NaN,NaN
2,NaN,FlashCo has grown to be one of the largest man...,NaN,NaN,NaN,NaN,NaN
3,NaN,PAC Worldwide is a global leader in the manufa...,NaN,NaN,NaN,NaN,NaN
4,NaN,"DynexÂ® Technologies, Inc. is an original pion...",NaN,NaN,NaN,NaN,NaN
5,NaN,"Auris Surgical Robotics, Inc. is a technology ...",NaN,NaN,NaN,NaN,NaN
6,NaN,Sense Corp powers insight-driven organizations...,NaN,NaN,NaN,NaN,NaN
7,NaN,We provide quality software and uncompromising...,NaN,NaN,NaN,NaN,NaN
8,NaN,Driven by the ever-changing needs of our clien...,NaN,NaN,NaN,NaN,NaN
9,NaN,"Americo Manufacturing Company, headquartered i...",NaN,NaN,NaN,NaN,NaN


Load spacy, a Natural Language Processing tool

In [3]:
import spacy

# proper english model needs to be loaded.
# nlp = spacy.load('en_core_web_sm')
spacy.__version__

ImportError: cannot import name 'prefer_gpu' from 'thinc.api' (C:\Users\avitr\anaconda3\lib\site-packages\thinc\api.py)

In [5]:
def tokenize_text(texts):
    docs = [doc for doc in nlp.pipe(texts, batch_size=500)]
    return docs

def to_text(tokens):
    return next(map(lambda token: token.orth_, tokens), '')

def filter_first_punct(noun_chunks):
    noun_chunks = list(noun_chunks)
    if len(noun_chunks) > 0:
        print('ROOT', noun_chunks[0].sent[noun_chunks[0].start])
    return []

def get_nouns(sentences):
    return [to_text(docs.noun_chunks) for docs in sentences]

Tokenize the titles and abstracts. Token types available [here](https://spacy.io/docs/usage/pos-tagging).

In [6]:
df['docs'] = tokenize_text(df['Title'].astype(str))

In [7]:
type(df['docs'][0])

spacy.tokens.doc.Doc

In [8]:
def print_tokens(article_docs):
    # print('Domain:       ', url)
    print('Title:     ', article_docs)
    print('-------------')
    print('Words:     ', list(map(lambda word: word, article_docs)))
    print('Lemma:     ', list(map(lambda word: word.lemma_, article_docs)))
    print('Types:     ', list(map(lambda word: word.pos_, article_docs)))
    print('Tags:      ', list(map(lambda word: word.tag_, article_docs)))
    print('>')
    print('Nouns:     ', list(filter(lambda word: word.pos_ == 'NOUN' or word.tag_ == 'NNP' or word.tag_ == 'NNPS', article_docs)))
    print('Nouns sentences (chunks):     ', get_nouns(article_docs.sents))
    print('Noun chunks:', list(article_docs.noun_chunks))
    print('Noun chunks +1 words:', list(filter(lambda chunk: len(str(chunk).split(' ')) >= 2, list(article_docs.noun_chunks))))
    print('Verbs:     ', list(filter(lambda word: word.pos_ == 'VERB', article_docs)))
    print('Verbs Lemma:', list(map(lambda word: word.lemma_, filter(lambda word: word.pos_ == 'VERB', article_docs))))
    print('Adjectives:', list(filter(lambda word: word.pos_ == 'ADJ', article_docs)))
    print('Adjs Lemma:', list(map(lambda word: word.lemma_, filter(lambda word: word.pos_ == 'ADJ', article_docs))))
    print('Adverbs:   ', list(filter(lambda word: word.pos_ == 'ADV', article_docs)))
    print('Adverbs Lemma:', list(map(lambda word: word.lemma_, filter(lambda word: word.pos_ == 'ADV', article_docs))))
    print('Superlatives:', list(filter(lambda word: word.tag_ == 'JJS' or word.tag_ == 'RBS', article_docs)))
    print('Entities:  ', list(map(lambda entity: (entity, entity.label_), article_docs.ents)))
    
def df_url_docs(id, docs_field = 'docs'):
    return df[docs_field][id]

Manually check that the correct data types have been identified

In [9]:
print_tokens(df_url_docs(0))

Title:      ParaDocs provides our clients with the highest level of emergency medical standby. Professionalism, discretion, and clinical excellence are our core values. From large public concerts to intimate private dinners we can provide an array of medical services to fit your needs. We provide only the most highly trained EMTâ€™s, paramedics, doctors, or ambulances services.
-------------
Words:      [ParaDocs, provides, our, clients, with, the, highest, level, of, emergency, medical, standby, ., Professionalism, ,, discretion, ,, and, clinical, excellence, are, our, core, values, ., From, large, public, concerts, to, intimate, private, dinners, we, can, provide, an, array, of, medical, services, to, fit, your, needs, ., We, provide, only, the, most, highly, trained, EMTâ€, ™, s, ,, paramedics, ,, doctors, ,, or, ambulances, services, .]
Lemma:      ['ParaDocs', 'provide', 'our', 'client', 'with', 'the', 'high', 'level', 'of', 'emergency', 'medical', 'standby', '.', 'professionalism

## Excel File Generation

In [10]:
# Loops through every article and applies f() to it. 
# Then applies token_extractor() to convert from a Token to a string.
# Finally, concatenates the tokens of a single type with commas
def filter_bad_excel_strings(tokens_string):
    tokens_string = re.sub('[\000-\010]|[\013-\014]|[\016-\037]', '', tokens_string)
    if tokens_string.startswith("="):
        return tokens_string[1:]
    elif tokens_string.startswith("- "):
        return tokens_string[2:]
    else:
        return tokens_string

def map_articles(token_extractor, f, articles):
    def map_article(article):
        tokens_string = ",".join(map(token_extractor, f(article)))
        # Replace excel invalid chars
        tokens_strings = filter_bad_excel_strings(tokens_string)
        return tokens_string
    return list(map(map_article, articles))

def make_excel_df(docs_column_name = 'docs'):
    df_excel = pd.DataFrame()
    docs = df[docs_column_name]
    df_excel['Title'] = df['Title']
    df_excel['Nouns'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'NOUN' or word.tag_ == 'NNP' or word.tag_ == 'NNPS', sentence), 
                                     docs)
    df_excel['Noun Chunks (1)'] = map_articles(lambda token: token, 
                                     lambda doc : get_nouns(doc.sents), 
                                     docs)
    df_excel['Noun Chunks (2)'] = map_articles(
                                     lambda chunk: str(chunk),
                                     lambda doc : list(doc.noun_chunks), 
                                     docs)
    df_excel['Noun Chunks (3) +1 words'] = map_articles(
                                     lambda chunk: str(chunk),
                                     lambda doc : list(filter(lambda chunk: len(str(chunk).split(' ')) >= 2, list(doc.noun_chunks))), 
                                     docs)
    #df_excel['Noun Chunks +1 words (3)'] = map_articles(lambda token: token, 
    #                                 list(filter(lambda chunk: len(str(chunk).split(' ')) >= 2, list(article_docs.noun_chunks))))
    df_excel['Verbs'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'VERB', sentence), 
                                     docs)
    df_excel['Verbs Lemma'] = map_articles(lambda token: token.lemma_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'VERB', sentence), 
                                     docs)
    df_excel['Adjectives'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'ADJ', sentence), 
                                     docs)
    df_excel['Adjectives Lemma'] = map_articles(lambda token: token.lemma_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'ADJ', sentence), 
                                     docs)
    df_excel['Adverbs'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'ADV', sentence), 
                                     docs)
    df_excel['Adverbs Lemma'] = map_articles(lambda token: token.lemma_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'ADV', sentence), 
                                     docs)
    df_excel['Superlatives'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.tag_ == 'JJS' or word.tag_ == 'RBS', sentence), 
                                     docs)
    df_excel['Superlatives Lemma'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.tag_ == 'JJS' or word.tag_ == 'RBS', sentence), 
                                     docs)
    df_excel['Entities'] = map_articles(lambda ent: ent.orth_, 
                                     lambda sentence : sentence.ents, 
                                     docs)
    return df_excel

In [11]:
df_excel_titles = make_excel_df()
df_excel_titles.head()

,Title,Nouns,Noun Chunks (1),Noun Chunks (2),Noun Chunks (3) +1 words,Verbs,Verbs Lemma,Adjectives,Adjectives Lemma,Adverbs,Adverbs Lemma,Superlatives,Superlatives Lemma,Entities
0,ParaDocs provides our clients with the highest...,"ParaDocs,clients,level,emergency,standby,Profe...","ParaDocs,Professionalism,large public concerts,We","ParaDocs,our clients,the highest level,emergen...","our clients,the highest level,emergency medica...","provides,intimate,provide,fit,provide,trained,...","provide,intimate,provide,fit,provide,train,amb...","highest,medical,clinical,large,public,private,...","high,medical,clinical,large,public,private,med...","only,most,highly","only,most,highly","highest,most","highest,most",ParaDocs
1,"AGNITY Global Inc., (AGNITY) is a global provi...","AGNITY,Global,Inc.,AGNITY,provider,Intelligent...","AGNITY Global Inc.,™s products leverage,All AG...","AGNITY Global Inc.,AGNITY,a global provider,In...","AGNITY Global Inc.,a global provider,Intellige...","™,enable,transform,become,capitalize,powered,l...","™,enable,transform,become,capitalize,power,lea...","global,real,workplace,competitive,new,secured,...","global,real,workplace,competitive,new,secured,...",more,more,,,"AGNITY Global Inc.,Intelligent Business Commun..."
2,FlashCo has grown to be one of the largest man...,"FlashCo,manufacturers,roof,flashings,accessori...","FlashCo,We,FlashCo,Our designs,FlashCo,FlashCo...","FlashCo,the largest manufacturers,roof flashin...","the largest manufacturers,roof flashings,North...","grown,be,providing,designed,made,consider,regi...","grow,be,provide,design,make,consider,register,...","largest,committed,lasting,available,significan...","large,committed,lasting,available,significant,...","best,longest,strongly,when","well,long,strongly,when","largest,best,longest,best,best,most,highest,ne...","largest,best,longest,best,best,most,highest,ne...","FlashCo,North America,FlashCo,FlashCo,FlashCo,..."
3,PAC Worldwide is a global leader in the manufa...,"PAC,Worldwide,leader,manufacturing,distributio...","PAC Worldwide,Our products,We","PAC Worldwide,a global leader,the manufacturin...","PAC Worldwide,a global leader,the manufacturin...","branded,include,provide,branded,receive","brand,include,provide,brand,receive","global,protective,flat,poly,highest,lasting,po...","global,protective,flat,poly,high,lasting,positive",now,now,highest,highest,
4,"DynexÂ® Technologies, Inc. is an original pion...","®,Technologies,Inc.,pioneer,technology,staff,p...","® Technologies,Our talented, multidisciplinary...","® Technologies,Inc.,an original pioneer,microp...","® Technologies,an original pioneer,microplate ...","include,manufacturing,deliver,cutting,meet,imp...","include,manufacture,deliver,cut,meet,improve,e...","original,microplate,talented,multidisciplinary...","original,microplate,talented,multidisciplinary...","approximately,worldwide,most,ultimately,also,w...","approximately,worldwide,most,ultimately,also,w...",most,most,"Technologies, Inc.,approximately 100,DSXÂ,DS2Â..."


In [12]:
writer = pd.ExcelWriter(output_filename)
df_excel_titles.to_excel(writer,'Titles')
writer.save()